In [1]:
import pandas as pd
import random
import numpy as np
from random import randint
import time
#import torch
#from transformers import AutoTokenizer, AutoModel

import time

import memory_profiler
%load_ext memory_profiler

from pathlib import Path

In [2]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [3]:
%load_ext autoreload
%autoreload 2

from pubmed_landscape_src.metrics import knn_accuracy_ls
from pubmed_landscape_src.data import generate_embeddings

In [4]:
variables_path = Path("../../results/variables")
figures_path = Path("../../results/figures")
berenslab_data_path = Path("/gpfs01/berens/data/data/pubmed_processed")

# Import

In [5]:
# Import
df = pd.read_pickle(berenslab_data_path / "df_labeled_papers_subset")
df = df.reset_index(drop=True)
abstracts = df["AbstractText"].tolist()

# Obtain embeddings

In [3]:
# random seed
random_state = random.seed(42)

In [4]:
# specify & check gpu usage
device = "cuda" if torch.cuda.is_available() else "cpu" # put cuda:0 if else not working
print("running on device: {}".format(device))

running on device: cuda


In [5]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('malteos/scincl')
model = AutoModel.from_pretrained('malteos/scincl')

print('model: malteos')

model: malteos


In [ ]:
%%capture cap
%%time

embeddings_cls = []
embeddings_sep = []
embeddings_av  = []

times = []

indeces_batches= np.arange(0, 1e6+1e2, 1e2, dtype=int)

for i in range(len(indeces_batches)-1):
    
    np.save(variables_path / 'experiment_iter', i)
    
    batch = abstracts[indeces_batches[i]:indeces_batches[i+1]]
    
    embedding_cls, embedding_sep, embedding_av = generate_embeddings(batch, tokenizer, model, device)
    
    name_embedding_cls = 'embedding_malteos/embedding_cls_batch_'+str(i)
    name_embedding_sep = 'embedding_malteos/embedding_sep_batch_'+str(i)
    name_embedding_av = 'embedding_malteos/embedding_cls_batch_'+str(i)
    
    np.save(berenslab_data_path / name_embedding_cls, embedding_cls, allow_pickle=True, fix_imports=True)
    np.save(berenslab_data_path / name_embedding_sep, embedding_sep, allow_pickle=True, fix_imports=True)
    np.save(berenslab_data_path / name_embedding_av, embedding_av, allow_pickle=True, fix_imports=True)

tcmalloc: large alloc 1258291200 bytes == 0xe74e0000 @ 
tcmalloc: large alloc 1258291200 bytes == 0x132956000 @ 


In [ ]:
with open( variables_path / 'verbose_batches_malteos.txt', 'w') as f:
    f.write(cap.stdout)

In [8]:
%%time
# join different batches

embeddings_cls = []
embeddings_sep = []
embeddings_av = []


indeces_batches = np.arange(0, 1e6 + 1e2, 1e2, dtype=int)

for i in range(len(indeces_batches) - 1):
    name_embedding_cls = (
        "embedding_malteos/embedding_cls_batch_" + str(i) + ".npy"
    )
    name_embedding_sep = (
        "embedding_malteos/embedding_sep_batch_" + str(i) + ".npy"
    )
    name_embedding_av = (
        "embedding_malteos/embedding_cls_batch_" + str(i) + ".npy"
    )

    embeddings_cls.append(
        np.load(
            berenslab_data_path / name_embedding_cls,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_sep.append(
        np.load(
            berenslab_data_path / name_embedding_sep,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_av.append(
        np.load(
            berenslab_data_path / name_embedding_av,
            allow_pickle=True,
            fix_imports=True,
        )
    )


embeddings_cls = np.vstack(embeddings_cls)
embeddings_sep = np.vstack(embeddings_sep)
embeddings_av = np.vstack(embeddings_av)

np.save(
    berenslab_data_path / "embeddings/embeddings_malteos/embeddings_cls",
    embeddings_cls,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    berenslab_data_path / "embeddings/embeddings_malteos/embeddings_sep",
    embeddings_sep,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    berenslab_data_path / "embeddings/embeddings_malteos/embeddings_av",
    embeddings_av,
    allow_pickle=True,
    fix_imports=True,
)

tcmalloc: large alloc 3072000000 bytes == 0x2eca30000 @ 
tcmalloc: large alloc 3072000000 bytes == 0x45b460000 @ 
tcmalloc: large alloc 3072000000 bytes == 0x512610000 @ 


CPU times: user 7.82 s, sys: 10.1 s, total: 17.9 s
Wall time: 25.4 s


# kNN accuracies

In [26]:
# Import
df = pd.read_pickle(berenslab_data_path / 'df_labeled_papers_subset')
df = df.reset_index(drop=True)
abstracts = df['AbstractText'].tolist()

In [27]:
labels = df['Colors'].tolist()

## CLS

In [ ]:
embeddings_cls = np.load(berenslab_data_path / 'embedding_malteos/embeddings_cls.npy', allow_pickle=True, fix_imports=True)

In [31]:
embeddings_cls.shape

(1000000, 768)

In [32]:
%%capture cap
%%time
knn_accuracy_malteos_cls = knn_accuracy_ls(embeddings_cls, labels)

tcmalloc: large alloc 3041280000 bytes == 0x5cd798000 @ 


In [33]:
with open( variables_path / 'verbose_knn_accuracy_malteos_cls.txt', 'w') as f:
    f.write(cap.stdout)

In [45]:
print(knn_accuracy_malteos_cls)

0.6526


In [34]:
np.save(variables_path / 'knn_accuracy_malteos_cls', knn_accuracy_malteos_cls)

## SEP

In [ ]:
embeddings_sep = np.load(berenslab_data_path / 'embedding_malteos/embeddings_sep.npy', allow_pickle=True, fix_imports=True)

In [35]:
embeddings_sep.shape

(1000000, 768)

In [36]:
%%capture cap
%%time
knn_accuracy_malteos_sep = knn_accuracy_ls(embeddings_sep, labels)

tcmalloc: large alloc 3041280000 bytes == 0x5cd798000 @ 


In [37]:
with open( variables_path / 'verbose_knn_accuracy_malteos_sep.txt', 'w') as f:
    f.write(cap.stdout)

In [46]:
print(knn_accuracy_malteos_sep)

0.6325


In [38]:
np.save(variables_path / 'knn_accuracy_malteos_sep', knn_accuracy_malteos_sep)

## Average

In [ ]:
embeddings_av = np.load(berenslab_data_path / 'embedding_malteos/embeddings_av.npy', allow_pickle=True, fix_imports=True)

In [39]:
embeddings_av.shape

(1000000, 768)

In [40]:
%%capture cap
%%time
knn_accuracy_malteos_av = knn_accuracy_ls(embeddings_av, labels)

tcmalloc: large alloc 3041280000 bytes == 0x5cd798000 @ 


In [41]:
with open( variables_path / 'verbose_knn_accuracy_malteos_av.txt', 'w') as f:
    f.write(cap.stdout)

In [47]:
print(knn_accuracy_malteos_av)

0.6536


In [42]:
np.save(variables_path / 'knn_accuracy_malteos_av', knn_accuracy_malteos_av)